# Proyecto Final - Aprendizaje Automático - Diego Estrada

## Problema de interés: Calidad de agua en el Río de la Plata

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

#Definimos una ruta base absoluta para levantar los 6 dataset
ruta_base = 'C:\\Users\\destrada\\Ciencia_datos_2A1C\\Cookiecutter_Proyecto_Final\\data\\interim\\'

#Verificamos si la ruta existe para evitar falla de localizacion de archivos, con la condicion IF validamos la existencia de ruta.
if os.path.exists(ruta_base):
    # Cargamos los datasets
    df1 = pd.read_csv(ruta_base + 'agc_y_riodelaplata_2017.csv', encoding='latin1', sep=';')
    df2 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2018.csv', encoding='latin1', sep=';')
    df3 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2019.csv', encoding='latin1', sep=';')
    df4 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2020.csv', encoding='latin1', sep=';')
    df5 = pd.read_csv(ruta_base + 'agc_z_riodelaplata_2021.csv', encoding='latin1', sep=';')
    df6 = pd.read_csv(ruta_base + 'agc_y_riodelaplata_2022.csv', encoding='latin1', sep=';')

    #Concatenamos los dataframes en uno solo Dataframe
    df_final = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
else:
    #En caso de no encontrar los archivos en la ruta, mostraria este mensaje.
    print("La ruta especificada no existe.")
    
#Mostramos las primeras filas del DataFrame unificado
df_final.head()

,Orden,Sitios,Gobierno_local,codigo,Fecha,Año,Campaña,Tem_agua,OD,pH,...,enteroc_ufc_100ml,P_de_ortof_mg_l,Hidr_Deriv_del_Petroleo_ug_l,Microcistina_ug_l,Clorofila_a_ug_l,ICA,Calidad_de_agua,Tem_aire,escherichia_coli_ufc_100ml,Hidrocarburos_Deriv_del_Petróleo_ug_l
0,1,Canal Villanueva y Río Luján,Tigre,TI001,8/3/2017,2017,verano,no se midió,6.01,7.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Río Lujan y Arroyo Caraguatá,Tigre,TI006,8/3/2017,2017,verano,no se midió,6.54,7.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Canal Aliviador y Río Lujan,Tigre,TI002,8/3/2017,2017,verano,no se midió,4.8,7.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Río Carapachay y Arroyo Gallo Fiambre,Tigre,TI003,8/3/2017,2017,verano,no se midió,5.89,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Río Reconquista y Río Lujan,Tigre,TI004,8/3/2017,2017,verano,no se midió,4.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Queremos conocer la cantidad de registros en el DataFrame Unificado
num_registros = df_final.shape[0]
print("Cantidad de registros: " + str(num_registros))

#Cantidad de columnas
num_columnas = df_final.shape[1]
print("Cantidad de columnas: " + str(num_columnas))

Cantidad de registros: 902
Cantidad de columnas: 39


##  Preprocesamiento de Datos (Data Preprocessing)
Vamos a trabajar en el proceso de ETL, después que unificamos los datos, eliminaremos algunas columnas que no necesitamos analizar, lo que también mejorará el rendimiento del modelo que vamos a desarrollar.

In [7]:
#Validamos si tenemos valores nulos en cada columnas
valores_nulos = df_final.isnull().sum()
print(valores_nulos)

Orden                                      0
Sitios                                     0
Gobierno_local                           744
codigo                                     0
Fecha                                      0
Año                                        0
Campaña                                    0
Tem_agua                                   9
OD                                       168
pH                                         3
colif_totales_ufc_100ml                   83
colif_fecales_ufc_100ml                  744
escher_coli_ufc_100ml                     83
Nitrato_mg_l                               4
NH4_mg_l                                   0
DQO_mg_l                                  82
DBO_mg_l                                   2
Fosf_ortofos_mg_l                        410
solidos_totales_mg_l                     744
Turbiedad_NTU                              0
Hidr_Deriv_Petr_ug_l                     414
Cr_total_mg_l                              5
Cd_total_m

In [15]:
#Vamos a quitar columnas que no serán relevante para el modelo de aprendizaje automatico
columnas_a_eliminar = ['Orden', 'Sitios', 'Gobierno_local', 'codigo', 'Fecha', 'Municipio','escherichia_coli_ufc_100ml',
                       'Hidrocarburos_Deriv_del_Petróleo_ug_l']

#Aplicamos la condición para validar
if all(col in df_final.columns for col in columnas_a_eliminar):
    df_final = df_final.drop(columns=columnas_a_eliminar)
else:
    print("Una o más columnas no existen.")

#Estantarizamos los nombre de las columnas
df_final.columns = df_final.columns.str.lower()

Una o más columnas no existen.


Luego de eliminar las columnas que no aportan información relevante para el análisis, se identificaron columnas que deberían contener exclusivamente valores numéricos de tipo flotante. Sin embargo además de presentar valores Nulos, algunas de estas columnas incluyen descripciones como 'No se midió'. Para garantizar la consistencia de los datos, estos valores serán reemplazados por la mediana correspondiente de cada columna.

In [17]:
#Con este ciclo, iteramos para que cada columna, reemplazamos valores y calcule la mediana
for col in ['od', 'ph', 'tem_agua','colif_totales_ufc_100ml','escher_coli_ufc_100ml','nitrato_mg_l','dqo_mg_l','dbo_mg_l','cr_total_mg_l',
            'colif_fecales_ufc_100ml','fosf_ortofos_mg_l','solidos_totales_mg_l','hidr_deriv_petr_ug_l','tem_aire','ica','enteroc_ufc_100ml',
            'p_de_ortof_mg_l','hidr_deriv_del_petroleo_ug_l']:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')
    df_final[col] = df_final[col].fillna(df_final[col].median())

Luego de haber tratado las columnas numéricas con valores nulos, reemplazándolos por la mediana correspondiente de cada columna, procederemos ahora a corregir los valores nulos paral las columnas categóricas.

In [33]:
#Columnas que queremos procesar
columnas = ['olores', 'color', 'espumas', 'mat_susp','microcistina_ug_l','clorofila_a_ug_l']

#Reemplazamos "no se midió" y valores nulos por la moda de cada columna
for columna in columnas:
    moda = df_final[columna].mode().iloc[0]  # Obtiene la moda (valor más frecuente)
    df_final[columna] = df_final[columna].replace("no se midió", pd.NA)
    df_final[columna] = df_final[columna].fillna(moda)

In [35]:
#Validamos si tenemos valores nulos en cada columnas
valores_nulos = df_final.isnull().sum()
print(valores_nulos)
num_registros = df_final.shape[0]
print("Cantidad de registros: " + str(num_registros))

año                               0
campaña                           0
tem_agua                          0
od                                0
ph                                0
colif_totales_ufc_100ml           0
colif_fecales_ufc_100ml           0
escher_coli_ufc_100ml             0
nitrato_mg_l                      0
nh4_mg_l                          0
dqo_mg_l                          0
dbo_mg_l                          0
fosf_ortofos_mg_l                 0
solidos_totales_mg_l              0
turbiedad_ntu                     0
hidr_deriv_petr_ug_l              0
cr_total_mg_l                     0
cd_total_mg_l                     0
p_total_l_mg_l                    0
olores                            0
color                             0
espumas                           0
mat_susp                          0
enteroc_ufc_100ml                 0
p_de_ortof_mg_l                   0
hidr_deriv_del_petroleo_ug_l      0
microcistina_ug_l                 0
clorofila_a_ug_l            